In [2]:
import os

In [3]:
import os
from google.cloud import bigquery
from google.cloud.bigquery import ScalarQueryParameter, QueryJobConfig
import pandas as pd

# 認証情報のパスを設定
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/atsuk/OneDrive/ドキュメント/GeminiCLI/2507 PatentsFinder/corded-guild-459506-i6-5bc162f91e16.json"

client = bigquery.Client()

# 検索条件
ipc_codes = ['B01D 61/00', 'G06N 20/00']
keywords = ['reverse osmosis membrane', 'machine learning', 'water filtration', 'membrane technology']
publication_date_from = '20160101'
# publication_date_toは指定なし（最大値で設定）
publication_date_to = '20250710'

# IPCコード条件
ipc_conditions = []
for i, ipc in enumerate(ipc_codes):
    ipc_conditions.append(f"EXISTS (SELECT 1 FROM UNNEST(p.ipc) AS ipc WHERE ipc.code LIKE @ipc_code_{i})")
ipc_sql = " OR ".join(ipc_conditions) if ipc_conditions else "1=1"

# キーワード条件
keyword_conditions = []
for i, kw in enumerate(keywords):
    keyword_conditions.append(f"LOWER((SELECT text FROM UNNEST(p.title_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1)) LIKE @keyword_{i}")
    keyword_conditions.append(f"LOWER((SELECT text FROM UNNEST(p.abstract_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1)) LIKE @keyword_{i}")
    keyword_conditions.append(f"LOWER((SELECT text FROM UNNEST(p.claims_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1)) LIKE @keyword_{i}")
keyword_sql = " OR ".join(keyword_conditions) if keyword_conditions else "1=1"

# IPCまたはキーワードのどちらかに一致
where_sql = f"({ipc_sql}) OR ({keyword_sql})"

sql = f"""
    SELECT
        p.publication_number,
        (SELECT text FROM UNNEST(p.title_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as title,
        (SELECT text FROM UNNEST(p.abstract_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as abstract,
        (SELECT text FROM UNNEST(p.claims_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as claims,
        (SELECT STRING_AGG(name) FROM UNNEST(p.assignee_harmonized)) as assignee,
        p.publication_date,
        (SELECT STRING_AGG(code) FROM UNNEST(p.ipc)) as ipc_codes
    FROM
        `patents-public-data.patents.publications` AS p
    WHERE
        p.publication_date >= @publication_date_from
        AND p.publication_date <= @publication_date_to
        AND ({where_sql})
    LIMIT 100
"""

query_params = [
    ScalarQueryParameter("publication_date_from", "INT64", int(publication_date_from)),
    ScalarQueryParameter("publication_date_to", "INT64", int(publication_date_to)),
]
for i, ipc_code in enumerate(ipc_codes):
    query_params.append(ScalarQueryParameter(f"ipc_code_{i}", "STRING", f"{ipc_code}%"))
for i, kw in enumerate(keywords):
    query_params.append(ScalarQueryParameter(f"keyword_{i}", "STRING", f"%{kw.lower()}%"))

job_config = QueryJobConfig(query_parameters=query_params)

print("--- BigQuery実行クエリ ---")
print(sql)
print("--- クエリパラメータ ---")
print(query_params)

query_job = client.query(sql, job_config=job_config)
df = query_job.result().to_dataframe()
print(f"{len(df)}件の特許が見つかりました。")
print(df.head())


--- BigQuery実行クエリ ---

    SELECT
        p.publication_number,
        (SELECT text FROM UNNEST(p.title_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as title,
        (SELECT text FROM UNNEST(p.abstract_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as abstract,
        (SELECT text FROM UNNEST(p.claims_localized) WHERE language = 'ja' OR language = 'en' LIMIT 1) as claims,
        (SELECT STRING_AGG(name) FROM UNNEST(p.assignee_harmonized)) as assignee,
        p.publication_date,
        (SELECT STRING_AGG(code) FROM UNNEST(p.ipc)) as ipc_codes
    FROM
        `patents-public-data.patents.publications` AS p
    WHERE
        p.publication_date >= @publication_date_from
        AND p.publication_date <= @publication_date_to
        AND ((EXISTS (SELECT 1 FROM UNNEST(p.ipc) AS ipc WHERE ipc.code LIKE @ipc_code_0) OR EXISTS (SELECT 1 FROM UNNEST(p.ipc) AS ipc WHERE ipc.code LIKE @ipc_code_1)) OR (LOWER((SELECT text FROM UNNEST(p.title_localized) WHERE lan

In [4]:
df

,publication_number,title,abstract,claims,assignee,publication_date,ipc_codes
0,US-2025054135-A1,Machine learning based evaluation of lateral f...,The present invention concerns computer-implem...,1 . A computer-implemented method for lateral ...,None,20250213,"G06V10/82,G06V10/77,G06V10/32,G06V10/26,G06V10..."
1,KR-102193381-B1,System and method for monitoring health of fan...,None,None,None,20201221,"G06N20/10,G05B23/02"
2,US-2025004982-A1,Non-homogeneous chiplets,A semiconductor module comprises multiple non-...,1 - 20 . (canceled) \n \n \n 2...,ADVANCED MICRO DEVICES INC,20250102,"G06T15/00,G06F15/76,G06F15/80"
3,US-2020320278-A1,Enhanced face-detection and face-tracking for ...,Embodiments described herein provide various e...,What is claimed is: \n \n 1 . A co...,ALTUMVIEW SYSTEMS INC,20201008,"G06T7/70,G06N5/04,G06K9/46,G06K9/00"
4,US-2022335072-A1,"Device, system and method for providing descri...","A device, system and method for providing desc...",What is claimed is: \n \n 1 . A me...,AMADEUS SAS,20221020,"G06F40/205,G06F16/335,G06N20/00,G06K9/62,G06F1..."
...,...,...,...,...,...,...,...
95,US-11734700-B2,Method and apparatus for determining data link...,This application relates to apparatus and meth...,What is claimed is: \n \n 1. A syste...,WALMART APOLLO LLC,20230822,"G06F9/44,G06F16/901,G06F17/18,G06F18/214,G06N2..."
96,US-2024144297-A1,Artificial intelligence based theme builder fo...,"Techniques are described herein for selecting,...",What is claimed is: \n \n 1 . A me...,WEVO INC,20240502,G06Q30/02
97,US-11885661-B2,Detecting material type using low-energy sensing,A system and method for detecting material typ...,What is claimed is: \n \n 1. A metho...,WILIOT LTD,20240130,"G01F23/28,G01F23/284,G01F23/296,G01N29/34,G01N..."
98,CN-207500679-U,A kind of water filtration water filling instr...,The utility model is related to machinery fiel...,None,XIAN INT INSTRUMENT MEASURE & CONTROL EQUIPMEN...,20180615,"F16L23/00,B01D35/02"
